In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition
import re
import string
from src.utils.recovery_analysis_utils import str_to_list

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

videos_data = pd.read_csv('data/sampled_decline_events_with_videos.csv')
videos_tag_data = pd.read_csv('data/videos_around_declines.csv')

videos_data['Videos_before'] = videos_data['Videos_before'].apply(str_to_list)
videos_data['Videos_after'] = videos_data['Videos_after'].apply(str_to_list)

In [3]:
# Create a data_frame with 2 index: the index of the decline and the source (before and after)

df_before = videos_data[['Videos_before']].explode('Videos_before')
df_before['Source'] = 'Before'
df_before = df_before.rename(columns={'Videos_before': 'Video'})

df_after = videos_data[['Videos_after']].explode('Videos_after')
df_after['Source'] = 'After'
df_after = df_after.rename(columns={'Videos_after': 'Video'})

df_combined = pd.concat([df_before, df_after], axis=0).reset_index()
df_combined = df_combined.set_index(['index', 'Source'])

df_combined.sort_values(by = ['index', 'Source'])
df_combined = df_combined.dropna()



In [4]:
# Merge to obtain the tags of all videos for each video before and after decline
df_combined['Tags'] = df_combined['Video'].map(lambda video: videos_tag_data.loc[video, 'tags'] if video in videos_tag_data.index else None)
df_combined


Video                                               Tags
index Source                                                            
0     Before  1684989                                         MsRosieBea
      Before  1684990  MsRosieBea,primark haul,primark haul august,pr...
      Before  1684991                                         MsRosieBea
      Before  1684992                                         MsRosieBea
      Before  1684993               MsRosieBea,red lip,get ready with me
...               ...                                                ...
36598 After   1889699  Music,beats,instrumental,right beat radio,stra...
      After   1889700  Music,beats,instrumental,right beat radio,late...
      After   1889701  Music,beats,instrumental,right beat radio,lofi...
      After   1889702  Music,beats,instrumental,right beat radio,mell...
      After   1889703  Music,beats,instrumental,right beat radio,lofi...

[2069978 rows x 2 columns]

In [7]:
#Get for each decline only 2 rows with the tags corresponding to the before and the after

df_tags_combined = df_combined.groupby(['index', 'Source'])['Tags'].apply(
    lambda x: list(set([item for sublist in x.dropna() for item in (sublist if isinstance(sublist, list) else [sublist])]))  # Handle NaN and non-list entries
).reset_index(name='Tags_combined')

# Filter to keep only one row for each 'index' and 'Source' combination
df_tags_combined = df_tags_combined[['index', 'Source', 'Tags_combined']]
df_tags_combined.set_index(['index', 'Source'], inplace=True)

df_tags_combined


Tags_combined
index Source                                                   
0     After   [MsRosieBea,back to uni,uni outfits, MsRosieBe...
      Before  [MsRosieBea,sunday, MsRosieBea, MsRosieBea,uni...
1     After   [hollow,generationhollow,the surge,surge,robo ...
      Before  [hollow,generationhollow,paragon,gameplay,alph...
2     After                                                  []
...                                                         ...
36595 Before  [Despacito accordion cover,Fonsi Despacito acc...
36597 After   [Shaper,Clapper,Keith Fenner,Fenner,machine sh...
      Before  [1985,Porsche 944,Wire Harness,early 1985 944 ...
36598 After   [Music,beats,instrumental,right beat radio,chi...
      Before  [Music,beats,instrumental,right beat radio,sou...

[61194 rows x 1 columns]